In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import os
import collections
import re 
from imageio import imread, imwrite
from multiprocessing import Pool
import matplotlib.pyplot as plt
import cv2
import gdown
#import matplotlib.pyplot as plt

## Check Converting Mask Code

In [3]:
cut_down_mapping = {
    0: 0,
    1: 0,
    2: 5,
    3: 0,
    4: 8,
    5: 8,
    6: 8,
    7: 1,
    8: 1,
    9: 1,
    10: 1,
    11: 4,
    12: 4,
    13: 4,
    14: 4,
    15: 4,
    16: 4,
    17: 5,
    18: 5,
    19: 5,
    20: 5,
    21: 6,
    22: 6,
    23: 7,
    24: 2,
    25: 2,
    26: 3,
    27: 3,
    28: 3,
    29: 3,
    30: 3,
    31: 3,
    32: 3,
    33: 3,
}
labels = {
    'unlabeled':  0 ,
    'ego vehicle':  1 ,
    'rectification border':  2 ,
    'out of roi' :  3 ,
    'static' :  4 ,
    'dynamic' :  5 ,
    'ground'    :  6 ,
    'road'      :  7 ,
    'sidewalk'  :  8 ,
    'parking'   :  9 ,
    'rail track': 10 ,
    'building'  : 11 ,
    'wall'      : 12 ,
    'fence'     : 13 ,
    'guard rail': 14 ,
    'bridge'    : 15 ,
    'tunnel'    : 16 ,
    'pole'      : 17 ,
    'polegroup' : 18 ,
    'traffic light': 19 ,
    'traffic sign'  : 20 ,
    'vegetation'    : 21 ,
    'terrain'       : 22 ,
    'sky'           : 23 ,
    'person'        : 24 ,
    'rider'         : 25 ,
    'car'           : 26 ,
    'truck'         : 27 ,
    'bus'           : 28 ,
    'caravan'       : 29 ,
    'trailer'       : 30 ,
    'train'         : 31 ,
    'motorcycle'    : 32 ,
    'bicycle'       : 33 ,
    'license plate' : -1 
}


color_map = {
    0: [0, 0, 0],
    1: [239, 234, 90],
    2: [44, 105, 154],
    3: [4, 139, 168],
    4: [13, 179, 158],
    5: [131, 227, 119],
    6: [185, 231, 105],
    7: [107, 137, 198],
    8: [241, 196, 83],
    9: [242, 158, 76],
    10: [234, 114, 71],
    11: [215, 95, 155],
    12: [207, 113, 192],
    13: [159, 89, 165],
    14: [142, 82, 172],
    15: [158, 115, 200], 
    16: [116, 95, 159],
}

In [4]:

#Reading an image, Simplifing it's labels to only 8 labels instead of 33 
def simplify_image_labels(image,viewable=False):
  new_image = np.zeros(image.shape)
  for k,v in cut_down_mapping.items():
    mask = image==k
    new_image[mask]= (255/8)*v if viewable else v
  return new_image



In [11]:
#filepath="./aachen_000056_000019_gtFine_labelIds.png"
filepath="./berlin_000000_000019_gtFine_labelIds.png"
og_filepath= "/no_backups/g013/data/gtFine_preprocessed/test/berlin/berlin_000000_000019_gtFine_labelIds.png"
output_path = "./output.png"
image = imread(filepath)
image_og = imread(og_filepath)
preprocessed_image=simplify_image_labels(image,False)
cv2.imwrite(output_path,preprocessed_image)

/tmp/ipykernel_2855808/410006855.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(filepath)
/tmp/ipykernel_2855808/410006855.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_og = imread(og_filepath)


True

In [12]:
for index1, arr in enumerate(image):
    for index2,value in enumerate(arr):
        converted_value= cut_down_mapping[value]
        value2 = preprocessed_image[index1][index2]
        value_og = image_og[index1][index2]
        if value2!=converted_value or value_og !=value2:
            print("There is a pixel that has a wrong value")

KeyboardInterrupt: 

In [203]:
image_res = imread(output_path)

/tmp/ipykernel_2356131/2657160487.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_res = imread(output_path)


In [16]:
#test_path="/no_backups/g013/data/gtFine_preprocessed/train/ulm/ulm_000063_000019_gtFine_labelIds.png"
test_path_2 =  "/no_backups/g013/data/v_2/train/aachen/aachen_000109_000019_gtFine_labelIds.png"
test_image = imread(test_path_2)
arr_unique =[]
for i in test_image:
    for j in i :
        arr_unique.append(j)
set(arr_unique)


/tmp/ipykernel_2855808/3274728154.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  test_image = imread(test_path_2)


{0, 1, 2, 3, 6, 8, 9, 11, 15}

## Downloading Extra Data from google drive

In [2]:
url = "https://drive.google.com/u/0/uc?id=1DtPo-WP-hjaOwsbj6ZxTtOo_7R_4TKRG"
PATH_CHECKPOINT = '/no_backups/g013/data/extra_data/refinement_final_v0.zip'
gdown.download(url, PATH_CHECKPOINT, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1DtPo-WP-hjaOwsbj6ZxTtOo_7R_4TKRG
To: /no_backups/g013/data/extra_data/refinement_final_v0.zip
100%|██████████| 5.19G/5.19G [01:15<00:00, 68.4MB/s]


'/no_backups/g013/data/extra_data/refinement_final_v0.zip'